In [1]:
from langchain.docstore.document import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import time
occupation = "politician"
CHROMA_SOURCE = "json_" + occupation
CHROMA_CHUNK_SIZE = 1000
CHROMA_PERSIST_DIRECTORY = "./chroma_db"
CHROMA_COLLECTION_NAME = occupation + "_json_files"
EMBEDDINGS_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
start_time = time.time()

target_collection_name = occupation + "_json_files"
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
print("Target Collection Is: " + target_collection_name)
chroma_db_from_disk = Chroma(persist_directory=CHROMA_PERSIST_DIRECTORY,                             
                            collection_name=target_collection_name,
                            embedding_function=embeddings)
results = chroma_db_from_disk._collection.get(include=["documents", "metadatas", "embeddings"])
documents = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(results["documents"], results["metadatas"])
]
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")
print("Number of Documents: "+  str(len(documents)))


/scratch/1030526/ipykernel_1362148/468504667.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
/home/hice1/tkaple3/.conda/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Target Collection Is: politician_json_files


/scratch/1030526/ipykernel_1362148/468504667.py:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db_from_disk = Chroma(persist_directory=CHROMA_PERSIST_DIRECTORY,


Execution time: 115.9676 seconds
Number of Documents: 101


In [3]:
documents[0]

Document(metadata={'first_name': 'Inoke', 'last_name': '', 'source': 'json_politician'}, page_content='["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\\n\\n### Instruction:\\nTranslate this English JSON File into a Hindi Language JSON File. \\n                Do not any other extra text or note after the conversion of JSON. \\n                Make sure everything in your response is in Hindi strictly and there are no additional NOTE afterwards\\n\\n### Input:\\n{\'_id\': \'Q58316\',\'sex or gender\': [\'male\'], \'instance of\': [\'human\'], \'occupation\': [\'diplomat\', \'politician\'], \'date of birth\': [{\'time\': \'+1948-06-16T00:00:00Z\', \'timezone\': 0, \'before\': 0, \'after\': 0, \'precision\': 11, \'calendarmodel\': \'http://www.wikidata.org/entity/Q1985727\'}], \'position held\': [\'Member of the House of Representatives of Fiji\', \'Member of th

In [19]:
import json
import ast
import time
from tqdm import tqdm
#all_keys = []
my_map = {}
errors = 0
for res in tqdm(documents):
    document = res  
    content = ast.literal_eval(document.page_content)[0]  # Parse the string as a Python literal and get the first element
    response_start = content.index("### Response:")
    json_start = content.index("{", response_start)
    json_end = content.rindex("}")
    json_string = content[json_start:json_end+1]
    try:
        parsed_dict = ast.literal_eval(json_string)
        for k in parsed_dict.keys():
            if k in my_map:
                my_map[k] += 1
            else:
                my_map[k] = 0
        #all_keys.append(json_keys)
        json_data = json.dumps(parsed_dict, ensure_ascii=False, indent=2)
        #print(json_data)
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing JSON: {e}")
        errors += 1
        continue

100%|██████████| 101/101 [00:00<00:00, 1701.52it/s]

Error parsing JSON: ':' expected after dictionary key (<unknown>, line 1)
Error parsing JSON: invalid character '–' (U+2013) (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: closing parenthesis '}' does not match opening parenthesis '[' (<unknown>, line 1)
Error parsing JSON: invalid character '०' (U+0966) (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: closing parenthesis '}' does not match opening parenthesis '[' (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was n

In [7]:
print(all_keys)

[dict_keys(['_id', 'लिंग या लिंग', 'उदाहरण', 'व्यवसाय', 'जन्म तिथि', 'स्थिति धारण की', 'सामान्य श्रेणी', 'नागरिकता का देश', 'छवि', 'दिया गया नाम', 'भाई-बहन', 'राजनीतिक दल का सदस्य']), dict_keys(['_id', 'लिंग', 'उदाहरण', 'जन्म की तिथि', 'व्यवसाय', 'जन्म स्थान', 'राजनीतिक दल', 'आयोजित पद', 'नागरिकता', 'संबंधित छवि', 'छवि', 'मृत्यु की तिथि']), dict_keys(['_id', 'लिंग या लिंग', 'उदाहरण', 'धर्म या विश्वदृष्टि', 'जन्म तिथि', 'व्यवसाय', 'छवि', 'नागरिकता', 'जन्म स्थान', 'पुरस्कार प्राप्त', 'शिक्षा प्राप्त की', 'कॉमन्स श्रेणी', 'परिवार नाम']), dict_keys(['_id', 'लिंग', 'जन्म की तारीख', 'उदाहरण', 'व्यवसाय', 'जन्म स्थान', 'नागरिकता', 'राजनीतिक दल के सदस्य', 'छवि', 'कॉमन्स श्रेणी', 'धारित पद', 'बोले जाने वाली भाषाएँ', 'स्वदेशी भाषा में नाम', 'पुरस्कार', 'शिक्षा', 'शैक्षिक उपाधि', 'आईएसएनआई', 'धर्म या विश्वदृष्टि', 'बच्चा']), dict_keys(['_id', 'लिंग या लिंग', 'उदाहरण', 'जन्म तिथि', 'राजनीतिक दल का सदस्य', 'व्यवसाय', 'दिया गया नाम', 'कॉमन्स श्रेणी', 'बच्चों की संख्या', 'छवि', 'जन्म स्थान', 'शिक्षित'

In [11]:
final_keys = []
for k in all_keys:
    final_keys.append(list(k))    

In [14]:
final_keys_flattened = [item for sublist in final_keys for item in sublist]

In [15]:
final_keys_flattened

['_id',
 'लिंग या लिंग',
 'उदाहरण',
 'व्यवसाय',
 'जन्म तिथि',
 'स्थिति धारण की',
 'सामान्य श्रेणी',
 'नागरिकता का देश',
 'छवि',
 'दिया गया नाम',
 'भाई-बहन',
 'राजनीतिक दल का सदस्य',
 '_id',
 'लिंग',
 'उदाहरण',
 'जन्म की तिथि',
 'व्यवसाय',
 'जन्म स्थान',
 'राजनीतिक दल',
 'आयोजित पद',
 'नागरिकता',
 'संबंधित छवि',
 'छवि',
 'मृत्यु की तिथि',
 '_id',
 'लिंग या लिंग',
 'उदाहरण',
 'धर्म या विश्वदृष्टि',
 'जन्म तिथि',
 'व्यवसाय',
 'छवि',
 'नागरिकता',
 'जन्म स्थान',
 'पुरस्कार प्राप्त',
 'शिक्षा प्राप्त की',
 'कॉमन्स श्रेणी',
 'परिवार नाम',
 '_id',
 'लिंग',
 'जन्म की तारीख',
 'उदाहरण',
 'व्यवसाय',
 'जन्म स्थान',
 'नागरिकता',
 'राजनीतिक दल के सदस्य',
 'छवि',
 'कॉमन्स श्रेणी',
 'धारित पद',
 'बोले जाने वाली भाषाएँ',
 'स्वदेशी भाषा में नाम',
 'पुरस्कार',
 'शिक्षा',
 'शैक्षिक उपाधि',
 'आईएसएनआई',
 'धर्म या विश्वदृष्टि',
 'बच्चा',
 '_id',
 'लिंग या लिंग',
 'उदाहरण',
 'जन्म तिथि',
 'राजनीतिक दल का सदस्य',
 'व्यवसाय',
 'दिया गया नाम',
 'कॉमन्स श्रेणी',
 'बच्चों की संख्या',
 'छवि',
 'जन्म स्थान',
 'शिक्

In [17]:
final_unique_keys = list(set(final_keys_flattened))

In [18]:
print(final_unique_keys)

['कॉपीराइट प्रतिनिधि', 'बालों की शैली/लंबाई', 'लिंग या लिंगिकता', 'बच्चे', 'बीएचसी यूआईडी', 'यूरोपीयना एंटिटी', 'पूर्वज', 'कमंस गैलरी', 'चुनावी उम्मीदवारी', 'दिया गया नाम', 'रुचि', 'कार्य में', 'रेजेन्सबर्ग वर्गीकरण', 'कार्यों का संग्रह', 'मंदिर नाम', 'संग्रह', 'कार्य काल (आरम्भ)', 'शिक्षित', 'हत्यारा', 'त्योहार दिवस', 'पति या पत्नी', 'यौन अभिविन्यास', 'शिक्षा प्राप्त', 'कार्य क्षेत्र', 'आवश्यक स्थिति', 'आवासित पद', 'चुनाव में उम्मीदवारी', 'शैक्षिक डिग्री', 'बपतिस्मा की तारीख', 'कॉमन्स क्रिएटर पेज', 'बायोग्राफी एट जर्मन बुंडेस्टैग यूआरएल', 'सैन्य', 'उदाहरण', 'स्रोत द्वारा वर्णित', 'शिक्षक', 'लिब्रिस-यूरआई', 'अंतिम विश्राम स्थान', 'धर्म या विश्वदृष्टि', 'जन्म का नाम', 'कार्य में उपस्थित', 'विकिमीडिया परियोजना पर ध्यान केंद्रित', 'एनएलसी प्राधिकरण', 'वर्डनेट 3', 'आदरणीय शीर्षक', 'अलग', 'अवस्थिति', 'Google Doodle', 'कब्र स्थान', 'मृत्यु की प्रक्रिया', 'अंतिम संस्कार की तारीख', 'नाम मूल भाषा में', 'मातृभाषा', 'संघर्ष', 'मृत्यु तिथि', 'U.S. National Archives Identifier', 'आधिकारिक वेबसाइट',

In [20]:
print(my_map)

{'_id': 79, 'लिंग या लिंग': 35, 'उदाहरण': 71, 'व्यवसाय': 79, 'जन्म तिथि': 48, 'स्थिति धारण की': 1, 'सामान्य श्रेणी': 25, 'नागरिकता का देश': 8, 'छवि': 50, 'दिया गया नाम': 63, 'भाई-बहन': 9, 'राजनीतिक दल का सदस्य': 15, 'लिंग': 29, 'जन्म की तिथि': 8, 'जन्म स्थान': 62, 'राजनीतिक दल': 26, 'आयोजित पद': 9, 'नागरिकता': 56, 'संबंधित छवि': 0, 'मृत्यु की तिथि': 5, 'धर्म या विश्वदृष्टि': 7, 'पुरस्कार प्राप्त': 31, 'शिक्षा प्राप्त की': 9, 'कॉमन्स श्रेणी': 38, 'परिवार नाम': 20, 'जन्म की तारीख': 19, 'राजनीतिक दल के सदस्य': 13, 'धारित पद': 9, 'बोले जाने वाली भाषाएँ': 0, 'स्वदेशी भाषा में नाम': 2, 'पुरस्कार': 5, 'शिक्षा': 37, 'शैक्षिक उपाधि': 1, 'आईएसएनआई': 20, 'बच्चा': 19, 'बच्चों की संख्या': 10, 'शिक्षित': 0, 'आवश्यक स्थिति': 4, 'सिबलिंग': 2, 'स्थिति धारित': 6, 'धार्मिक विश्वास': 6, 'विकिमीडिया परियोजना की फोकस सूची पर': 0, 'बोली जाने वाली भाषाएं': 0, 'पिता': 19, 'लिंग या लैंगिक': 3, 'मृत्यु की तारीख': 13, 'परिवार': 11, 'मृत्यु स्थान': 31, 'वर्णित स्रोत': 27, 'समाधि स्थल': 10, 'सिब्लिंग': 1, 'भाषाएँ ब

In [23]:
sorted_dict = dict(sorted(my_map.items(), key=lambda item: item[1], reverse = True))

In [24]:
print(sorted_dict)

{'_id': 79, 'व्यवसाय': 79, 'उदाहरण': 71, 'दिया गया नाम': 63, 'जन्म स्थान': 62, 'नागरिकता': 56, 'छवि': 50, 'जन्म तिथि': 48, 'परिवार का नाम': 41, 'कॉमन्स श्रेणी': 38, 'शिक्षा': 37, 'लिंग या लिंग': 35, 'ISNI': 33, 'पुरस्कार प्राप्त': 31, 'मृत्यु स्थान': 31, 'लिंग': 29, 'वर्णित स्रोत': 27, 'मृत्यु तिथि': 27, 'विषय की मुख्य श्रेणी': 27, 'राजनीतिक दल': 26, 'हस्ताक्षर': 26, 'सामान्य श्रेणी': 25, 'मूल भाषा में नाम': 25, 'सदस्य': 22, 'परिवार नाम': 20, 'आईएसएनआई': 20, 'मृत्यु का कारण': 20, 'जन्म की तारीख': 19, 'बच्चा': 19, 'पिता': 19, 'मूल नाम': 19, 'पत्नी': 19, 'चित्र': 17, 'कॉमन्स गैलरी': 17, 'माता': 16, 'संघर्ष': 16, 'स्रोत द्वारा वर्णित': 16, 'एनई': 16, 'राजनीतिक दल का सदस्य': 15, 'जीवनसाथी': 15, 'नियोक्ता': 15, 'बोली जाने वाली भाषाएँ': 15, 'मूल भाषा': 14, 'राजनीतिक दल के सदस्य': 13, 'मृत्यु की तारीख': 13, 'आधिकारिक वेबसाइट': 13, 'भाषाएँ': 12, 'परिवार': 11, 'बच्चों की संख्या': 10, 'समाधि स्थल': 10, 'कार्य स्थान': 10, 'NE': 10, 'भाई-बहन': 9, 'आयोजित पद': 9, 'शिक्षा प्राप्त की': 9, 'धारित पद':

In [35]:
selected_keys = [f"{{{{{key}}}}}" for key, value in sorted_dict.items() if value == 0]

In [36]:
selected_keys

['{{संबंधित छवि}}',
 '{{बोले जाने वाली भाषाएँ}}',
 '{{शिक्षित}}',
 '{{विकिमीडिया परियोजना की फोकस सूची पर}}',
 '{{बोली जाने वाली भाषाएं}}',
 '{{भाषाएँ बोली या लिखी या संकेत}}',
 '{{धार्मिक विश्वास या दृष्टिकोण}}',
 '{{कनोनाइजेशन स्थिति}}',
 '{{त्योहार दिवस}}',
 '{{भाषाएँ बोली गईं, लिखी गईं या संकेतित}}',
 '{{कब्र की चित्र}}',
 '{{कब्र स्थान}}',
 '{{संग्रहालय में}}',
 '{{विकिमीडिया प्रोजेक्ट पर फोकस लिस्ट}}',
 '{{बीएचसी यूयूआईडी}}',
 '{{योगदान क्रियेटिव वर्क}}',
 '{{विकिप्रोजेक्ट द्वारा बनाए रखा गया}}',
 '{{बोली गई भाषा}}',
 '{{NSDAP सदस्यता संख्या (1925–1945)}}',
 '{{बंदी स्थल}}',
 '{{कार्य काल (आरम्भ)}}',
 '{{मुंहासे}}',
 '{{यौन अभिमुखता}}',
 '{{विकिमीडिया परियोजना पर फोकस सूची में}}',
 '{{आदरणीय शीर्षक}}',
 '{{मालिक}}',
 '{{सैन्य/पुलिस पद}}',
 '{{इमेज आर्काइव, हर्डर इंस्टीट्यूट}}',
 '{{स्वीडिश ओपन कल्चरल हेरिटेज यूआरआई}}',
 '{{कार्यों में है}}',
 '{{निर्माता के रूप में कॉपीराइट स्थिति}}',
 '{{उत्पत्ति देश}}',
 '{{भाषाएँ बोली लिखी या चिन्हित}}',
 '{{नोबल शीर्षक}}',
 '{{बायोग्राफी एट ज

In [12]:
final_list = []
for doc in documents:
    my_metadata = doc.metadata
    first_name = my_metadata["first_name"]
    last_name = my_metadata["last_name"]
    if first_name == "" or last_name == "":
        continue
    else:
        final_list.append([first_name, last_name])
print(final_list)

[['Wilfred', 'Elrington'], ['Ernst', 'Siemens'], ['Erich', 'Ludendorff'], ['Ernst', 'Röhm'], ['Daniel', 'Romanowitsch'], ['Franz', 'Müntefering'], ['Hans', 'Fritzsche'], ['Karl', 'Gebhardt'], ['Robert', 'Ley'], ['Otto', 'Braun'], ['Oskar', 'Dirlewanger'], ['Mikhail', 'Lomonosov'], ['Hanns', 'Schleyer'], ['Sandra', 'Perković'], ['Friedrich', 'von Humboldt'], ['Ludwig', 'Grimm'], ['Jean-Marie', 'Le Pen'], ['Zheng', 'Ying'], ['Rosa', 'Luxemburg'], ['Giacomo', 'Puccini'], ['Giuseppe', 'Verdi'], ['Jacques', 'Cartier'], ['Yury', 'Gagarin'], ['Lun', 'Sima'], ['Margaret', 'Roberts'], ['Jian', 'Yang'], ['Nina', 'Andreeva'], ['Irène', 'Joliot-Curie'], ['Willoughby', 'Newton'], ['Charles', 'Maurras'], ['Guy', 'Teissier'], ['William', 'Terry'], ['James', 'Gibson'], ['Allen', 'Sheppard'], ['Elisha', 'Meredith'], ['Henry', 'Alexander'], ['Todd', 'Platts'], ['David', 'Cairns'], ['Marit', 'Paulsen'], ['Michael', 'Shaw'], ['John', 'Gregg'], ['Leonid', 'Razvozzhayev'], ['Mohamed', 'Sheikh'], ['Albert', 

In [13]:
len(final_list)

78